In [1]:
import numpy as np
from numba import jit
from time import time_ns

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd

@jit
def exp(a:float):
    return np.exp(a)

class State():
    beta = 10
    def __init__(self, size:int=10):
        self.size = size
        self.value = np.array([False] * size)
        self.inter = np.array([[float(0)]*size]*size)
        self.field = np.array([float(0)]*size)
        self.flip_pro = np.array([float(0)]*size)
        self.N = 1
        self.goal = np.array([[False]*size]*self.N)

    def __str__(self):
        return ("Values\n "+str(list(map(int, self.value)))
        +"\nInteractions\n"+str(self.inter)+"\nField\n "+str(self.field))

    def setInterByIndex(self, i:int, j:int, value:float):
        try:
            self.inter[i][j] = value
        except IndexError:
            print("Index Out of Range: "+ str(i)+', '+str(j))
        return value
    def setInter(self, li: np.array):
        if len(li) != self.size or len(li[0])!= self.size:
            print("SetInter Data Size Error")
            return
        self.inter = li
        return
    def setValue(self, li: np.array):
        if len(li) != self.size:
            print("SetValues Data Size Error")
            return
        self.value = li
        return
    def setField(self, li: np.array):
        if len(li) != self.size:
            print("SetField Data Size Error")
            return
        self.field = li
        return
    def hamiltonian(self):
        return sum(sum(pow((self.value*self.inter).T*self.value, 1))) + sum(self.value*self.field)
    def roll(self):
        self.value = np.random.randint(0, 2, size=self.size, dtype=bool)


    def calcField(self):
        ans = list()
        for i in range(self.size):
            ans.append((self.inter[i]*self.value).sum())
        self.field = np.array(ans)
        return self.field

    def calcFlip(self):
        Del_H = self.field - self.field*self.value*2
        self.flip_pro = np.exp(-self.beta*Del_H)
        self.flip_pro = self.flip_pro / self.flip_pro.sum()
        # if Del_H<0: self.flip_pro=1
        # else: self.flip_pro=0
        return self.flip_pro

    def updateField(self, index:int):
        
        return self.field

    def step(self):
        # self.calcField() # N^2 Replaced with updatefield
        start = (time_ns())
        n = np.random.randint(0, self.size)
        val = self.value[n]
        field = self.field[n]
        # print(1, time_ns()-start)
        if (val and field<0) or ((not val) and field>0):
            self.value[n] = not self.value[n]
            # print(2, time_ns()-start)
            self.field += self.inter.T[n] * (2-4*self.value[n])
            # print(3, time_ns()-start)
        else:
            p = np.exp(-self.beta*field*(val*2-1))
            # print(4, time_ns()-start)
            
            if np.random.uniform() < p:
                self.value[n] = not self.value[n]
                self.field += self.inter.T[n] * (2-4*self.value[n])
            # print(5, time_ns()-start)
        return self.value

    def setGoal(self, goal:np.ndarray, option="hopfield"):
        self.N = len(goal)
        self.goal = goal
        if option == "hopfield":
            self.hopfield_interaction()
        return self.inter

    def hopfield_interaction(self):
        self.field = np.array([float(0)]*self.size)
        self.inter += np.dot((self.goal*2-1).T, (self.goal*2-1))/self.N
    
    def eval_interaction(self, flip_number=10, steps=1000):
        dataset=self.goal
        modified = [flip(data, flip_number=flip_number) for data in dataset]
        result = []
        for (index, start) in enumerate(modified):
            self.setValue(start)
            for _ in range(steps):
                if(_%100==0): self.calcField()
                self.step()
            result.append((self.value^dataset[index]).sum())
        return np.array(result).mean()
        
    def small_adj(self, learning_rate=0.1, delta=0.1):
        loss0 = self.eval_interaction()
        loss_difference = np.array([[float(0)]*self.size]*self.size)
        for i in range(self.size):
            for j in range(self.size):
                self.inter[i][j] += delta
                loss1 = self.eval_interaction()
                self.inter[i][j] -= delta
                loss_difference[i][j] = (loss1-loss0)/delta
        self.inter -= loss_difference * learning_rate
        return self.eval_interaction()

    def plot_loss(self, flip_number=10, steps=1000, count=100):
        data = [self.eval_interaction(flip_number=flip_number, steps=steps) for _ in range(count)]
        # sns.histplot(
        #     data
        # )
        return data

def flip(input: np.ndarray, flip_number:int) :
    arr = input.copy()
    for index in np.random.randint(0, len(arr), dtype=int, size=flip_number):
        arr[index] = not arr[index]
    return arr


In [3]:
size = 200; modification = 10
inter = np.array([[float(0)]*size]*size)
dataset=np.array([np.random.randint(0, 2, size=size, dtype=bool) for _ in range(20)])
state = State(size)
state.setGoal(dataset)
rand = np.random.uniform

answer_inter = state.inter

damage_edge = pd.DataFrame()
damage_edge_numbers = range(0, 20200, 200)
for n in damage_edge_numbers:
    temp = answer_inter
    for i in np.random.choice(size*size, n):
        temp[int(i/size)][i%size] = 0
    state.setInter(temp)
    damage_edge[n]=state.plot_loss()
    sns.histplot(
        damage_edge[n]
    ).get_figure().savefig(f"damaged_edge/edge{str(n)}.png")
    plt.close()
    damage_edge.to_csv(f"damaged_edge/damaged_edge.csv")
    print(n)


0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
13800
14000
14200
14400
14600
14800
15000
15200
15400
15600
15800
16000
16200
16400
16600
16800
17000
17200
17400
17600
17800
18000
18200
18400
18600
18800
19000
19200
19400
19600
19800
20000


<ipython-input-3-ce2fbfaaa86c>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  damage_edge[n]=state.plot_loss()


In [ ]:
from scipy.optimize import minimize

size = 200; modification = 10
inter = np.array([[float(0)]*size]*size)
dataset=np.array([np.random.randint(0, 2, size=size, dtype=bool) for _ in range(20)])
state = State(size)
state.setGoal(dataset)
rand = np.random.uniform

answer_inter = state.inter

damage_node = pd.DataFrame()
for n in range(11):
    temp = answer_inter
    for i in np.random.choice(size, n):
        for j in range(size):
            temp[i][j] = 0
            temp[j][i] = 0
    state.setInter(temp)
    damage_node[n]=state.plot_loss()
    sns.histplot(
        damage_node[n]
    ).get_figure().savefig(f"damaged_node/node{str(n)}.png")
    plt.close()
    plt.close()
    damage_node.to_csv(f"damaged_node/damaged_node.csv")

In [ ]:
import tensorflow.keras.optimizers as optimizers

size = 200; modification = 10
inter = np.array([[float(0)]*size]*size)
dataset=np.array([np.random.randint(0, 2, size=size, dtype=bool) for _ in range(10)])
state = State(size)
state.setGoal(dataset, option='hopfield')

def loss():
    a = state.eval_interaction(flip_number=0)
    print(a)
    return a

opt = optimizers.Adam(
    learning_rate = 0.01
)
opt_op = opt.minimize(loss, var_list=(state.inter.flatten()))
opt_op.run()

In [ ]:
from scipy.optimize import minimize
import numpy as np

def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

In [ ]:
def func():
    size = 200
    dataset=np.array([np.random.randint(0, 2, size=size, dtype=bool) for _ in range(10)])
    modified = [flip(data, 10) for data in dataset]
    a = State(size)
    a.setGoal(dataset, option='hopfield')
    print(a.eval_interaction())
    for _ in range(10):
        print(a.small_adj())

func()

In [ ]:
size = 1000
dataset=np.array([np.random.randint(0, 2, size=size, dtype=bool) for _ in range(10)])
modified = [flip(data, 10) for data in dataset]
a = State(size)
a.setGoal(dataset)
# print(dataset[0])
# print(a.inter)
temp = 0
for (index, start) in enumerate(modified):
    a.setValue(start)
    for _ in range (1000):
        a.step()
    ans = (a.value^dataset[index]).sum()
    print(ans, end =' ')
    temp = a

In [ ]:
dataset[0] ^ a.value

In [ ]:
a.calcField()

In [ ]:
dataset=np.array([np.random.randint(0, 2, size=N, dtype=bool) for _ in range(100)])
modified = [flip(data, 0) for data in dataset]
len(dataset)

In [ ]:
a=State(2)
inter = [[1, 2], [3, 4]]
inter = np.array(inter, dtype=float)
print(inter)
a.setInter(inter)
a.setValue(np.array([True, False]))
a.setField(np.array([100, 4], dtype=float))
a.roll()
print(a.value)
print(a.hamiltonian())

In [ ]:
# Hopfield Network
def HopfiledNetwork():
    pass

In [ ]:
def POWER(a, b):
    